<a href="https://colab.research.google.com/github/miam-bonbon/assignment-adv-nlp/blob/main/assignment_computer_vision_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Computer Vision - Fruit Detection with yolov5

by Michael Hunziker

## Setting up the YOLOv5 environment

### Connect to Google Drive

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Install and start comet

In [2]:
!pwd

/content


In [3]:
%cd /content/drive/MyDrive/assignment-computer-vision/
!ls -l

/content/drive/MyDrive/assignment-computer-vision
total 73436
drwx------ 2 root root     4096 Nov 23 13:30 data
drwx------ 2 root root     4096 Nov 26 16:28 datasets
drwx------ 2 root root     4096 Nov 26 15:35 fruit-detection-yolov5
-rw------- 1 root root   363112 Nov 26 11:53 model.png
-rw------- 1 root root     7733 Nov 23 13:33 plot_functions.py
drwx------ 2 root root     4096 Nov 23 13:43 __pycache__
drwx------ 2 root root     4096 Nov 26 06:52 raw_test_ds_one_banana_only_saved
drwx------ 2 root root     4096 Nov 25 19:35 raw_test_ds_saved
drwx------ 2 root root     4096 Nov 26 06:52 raw_train_ds_one_banana_only_saved
drwx------ 2 root root     4096 Nov 24 19:18 raw_train_ds_saved
drwx------ 2 root root     4096 Nov 26 06:52 raw_validation_ds_one_banana_only_saved
drwx------ 2 root root     4096 Nov 24 19:19 raw_validation_ds_saved
-rw------- 1 root root  5613764 Nov 26 16:30 yolo11n.pt
-rw------- 1 root root 50589507 Nov 26 20:24 yolov5mu.pt
-rw------- 1 root root 18581255 Nov 26

In [4]:
%pip install comet_ml --quiet

### Initialize Comet

We have to initialize and import before any references to torch

In [5]:
import comet_ml

from comet_ml import start
from comet_ml.integration.pytorch import log_model

comet_ml.login()
experiment = start(
  project_name="fruit-detection-yolov5",
  workspace="miam-bonbon"
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/miam-bonbon/fruit-detection-yolov5/1ed64d4c04bb4f3e9a7d687f512e0fb3

COMET INFO: Couldn't find a Git repository in '/content/drive/MyDrive/assignment-computer-vision' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


### Install ultralytics in our Colab environment:

In [6]:
!pip install torch torchvision torchaudio

In [7]:
!pip install ultralytics

In [8]:
# %%capture

# # %load_ext autoreload
# # %autoreload 2
# import os
# import glob
# import random

# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import tqdm.auto as tqdm

# import tensorflow as tf

In [9]:
# import plot_functions

In [10]:
!nvidia-smi
# tf.config.list_physical_devices('GPU')

Wed Nov 27 19:40:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## Setup the data

We already have our data set up in the right folders on Google Drive

In [11]:
# %cd data_yolov5
!ls datasets/data_yolov5 -l

total 18160
-rw------- 1 root root      347 Nov 26 16:28 data.yaml
-rw------- 1 root root      148 Nov 26 13:01 README.dataset.txt
-rw------- 1 root root      990 Nov 26 13:01 README.roboflow.txt
drwx------ 2 root root     4096 Nov 26 13:14 test
drwx------ 2 root root     4096 Nov 26 13:14 train
drwx------ 2 root root     4096 Nov 26 13:14 valid
-rw------- 1 root root 18581255 Nov 26 14:01 yolov5su.pt


In [12]:
import os

# set up our data paths
base_path = 'datasets/data_yolov5'
train_image_dir = os.path.join(base_path, 'train')
validation_image_dir = os.path.join(base_path, 'valid')
test_image_dir = os.path.join(base_path, 'test')
yaml_config = os.path.join(base_path, 'data.yaml')

## Training

In [17]:
from ultralytics import YOLO

# let's evaluate with 5s
# model = YOLO('yolov5s.pt')
model = YOLO('yolov5m.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
PRO TIP 💡 Replace 'model=yolov5m.pt' with new 'model=yolov5mu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 48.2M/48.2M [00:00<00:00, 187MB/s]


In [18]:
import os

config_file_path = yaml_config  # Replace with your config.yaml path

if os.path.exists(config_file_path):
    print("config.yaml found at:", config_file_path)
else:
    print("config.yaml not found at:", config_file_path)

config.yaml found at: datasets/data_yolov5/data.yaml


In [21]:
# Train the model with Comet integration
results = model.train(
    data=yaml_config,
    batch=16,
    epochs=100,
    project='fruit-detection-yolov5',  # Your Comet project name
    name='fruit-detection-5m',  # Your experiment name
    # verbose=True,
    cache=True,
)

Ultralytics 8.3.38 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov5m.pt, data=datasets/data_yolov5/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=fruit-detection-yolov5, name=fruit-detection-5m3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=Fa

train: Scanning /content/drive/MyDrive/assignment-computer-vision/datasets/data_yolov5/train/labels.cache... 2205 images, 3 backgrounds, 0 corrupt: 100%|██████████| 2205/2205 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (2.5GB RAM): 100%|██████████| 2205/2205 [00:06<00:00, 331.86it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

     44/100      7.17G     0.3785     0.2638     0.9129         76        640: 100%|██████████| 138/138 [00:17<00:00,  7.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:02<00:00,  6.09it/s]

                   all        448       1132      0.978      0.824      0.819      0.721



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     45/100      7.17G     0.3705     0.2585     0.9086         74        640:  29%|██▉       | 40/138 [00:05<00:12,  7.64it/s]


KeyboardInterrupt: 

In [22]:
!python --version
!ls -l

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [23]:
# save to comet
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : financial_glazing_8095
COMET INFO:     url                   : https://www.comet.com/miam-bonbon/fruit-detection-yolov5/bf4ead2c50014a1d9ecfc784d348af2c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [611]                : (20750.0859375, 6440967.0)
COMET INFO:     lr/pg0 [88]               : (0.00033091787439613527, 0.0009778323671497586)
COMET INFO:     lr/pg1 [88]               : (0.00033091787439613527, 0.0009778323671497586)
COMET INFO:     lr/pg2 [88]               : (0.00033091787439613527, 0.0009778323671497586)
COMET INFO:     metrics/mAP50(B) [88]     : (0.72696, 0.82665)
COMET INFO:     metrics/mAP50-95(B) [88]  : (0.59352,